In [67]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [123]:
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 크롬 드라이버 경로 설정
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.yogiyo.co.kr/')
time.sleep(0.1)  # 페이지 로딩 대기 (필요 시 더 길게 조정)

elem = driver.find_element(By.CLASS_NAME, 'btn.btn-default.ico-loc')
elem.click()
print(elem)
driver.implicitly_wait(10)

<selenium.webdriver.remote.webelement.WebElement (session="5818f3046a4fc5619633966ae29b7d47", element="f.48918E436D2C65418233F60B2CC444F0.d.389A89D2C6714041A6DDF91D860CCDAE.e.67")>


In [134]:
# 따로 땔 줄 몰라서 일단 분리 실행
# elem = driver.find_elements(By.CLASS_NAME, 'category-name')
# print(len(elem))
# print(elem[1])

# time.sleep(0.1)
from selenium.webdriver.support.ui import Select


category_items = driver.find_elements(By.CLASS_NAME, "category-name")
for item in category_items:
    if "치킨" in item.text:
        print("클릭 대상:", item.text)
        item.click()
        break
driver.implicitly_wait(10)

select_element = driver.find_element(By.CSS_SELECTOR, "select.form-control")

# Selenium의 Select 객체로 감싸기
select = Select(select_element)

# '리뷰 많은순'을 선택 (value 또는 visible text로 가능)
select.select_by_value("review_count")

# for item in category_items:
#     print(item.text)
time.sleep(0.4)
driver.implicitly_wait(10)

elem = driver.find_elements(By.CLASS_NAME, 'item.clearfix')
print(len(elem))
test = []
for a in elem:
    test.append(a.text.split('\n')[0])

클릭 대상: 치킨
60


In [141]:
print(len(test))
print(len(set(test)))
from itertools import zip_longest  

for a, b in zip_longest(test, set(test)) :
    if(a.text.split('\n')[0] != b.text.split('\n')[0]):
        print(a.text.split('\n')[0])

60
56


AttributeError: 'str' object has no attribute 'text'

In [102]:
# 렌더링된 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 이미지 div 추출
photo_divs = soup.find_all('div', class_='photo')
menu = soup.find_all('div', class_='menu-name')

for div_p, div_m in zip(photo_divs, menu):
    style = div_p.get('style', '')
    image_urls = re.findall(r"url\('([^']+)'\)", style)
    print(image_urls[0])
#     print(div_m.text)
#     for idx, image_url in enumerate(image_urls):
#         print(f"이미지 URL: {image_url}")
# driver.quit()

In [43]:
print(len(set(photo_divs)))
len(set(menu))

74


77

In [52]:
# for a in set(menu) :
#     print(a.text)
    
for i, div_p in enumerate(set(photo_divs)) :
    style = div_p.get('style', '')
    image_urls = re.findall(r"url\('([^']+)'\)", style)
    print(image_urls[0])
    if(extract_menu_name(image_urls[0]) == "메뉴명 추출 실패"):
        print(i)
        print(image_urls[0])
    

https://images.yogiyo.co.kr/image/yogiyo/PARTNER_FR_IMG/%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90/2024-07-09/%EC%A0%9C%ED%9C%B4FR_20240709_%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90_%ED%8C%8C%EC%9D%B8%EC%95%A0%ED%94%8C%EC%B6%94%EA%B0%80_1080x640.jpg?width=384&height=273&quality=100
https://images.yogiyo.co.kr/image/yogiyo/PARTNER_FR_IMG/%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90/2024-07-09/%EC%A0%9C%ED%9C%B4FR_20240709_%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90_%ED%8E%98%ED%8E%98%EB%A1%9C%EB%8B%88%EC%B6%94%EA%B0%80-20%EA%B0%9C_1080x640.jpg?width=384&height=273&quality=100
https://images.yogiyo.co.kr/image/yogiyo/PARTNER_FR_IMG/%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90/2024-07-09/%EC%A0%9C%ED%9C%B4FR_20240709_%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90_%EA%B0%88%EB%A6%AD%EB%8F%84%EA%B7%B8%EC%8A%A4%ED%8B%B1_1080x640.jpg?width=384&height=273&quality=100
https://images.yogiyo.co.kr/image/yogiyo/PARTNER_FR_IMG/%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90/2024-07-09/%EC%A0%9C%ED%9C%B4FR_20240709_%EC%B2%AD%EB%85%84%ED%94%BC%EC%9E%90_%

In [35]:
from urllib.parse import unquote, urlparse
import re

def extract_menu_name(url):
    # URL 경로 부분만 가져오기
    path = urlparse(url).path  # 예: /image/.../제휴FR_20240709_청년피자_슈퍼슈프림_1080x640.jpg

    # 한글로 디코딩
    decoded_path = unquote(path)

    # 파일명 추출
    filename = decoded_path.split('/')[-1]  # 제휴FR_20240709_청년피자_슈퍼슈프림_1080x640.jpg

    # 정규표현식으로 메뉴 이름 추출
    match = re.search(r'청년피자_(.+?)_1080x640', filename)
    if match:
        return match.group(1)
    else:
        return None


In [42]:
from urllib.parse import unquote, urlparse
import re
from itertools import zip_longest

# 메뉴명 추출 함수
def extract_menu_name(url):
    path = urlparse(url).path
    decoded = unquote(path)
    filename = decoded.split('/')[-1]
    match = re.search(r'청년피자_(.+?)_1080x640', filename)
    return match.group(1) if match else "메뉴명 추출 실패"

# 비교 시작
from itertools import zip_longest

for i, (a, div_p) in enumerate(zip_longest(menu, photo_divs)):
    menu_text = a.text.strip() if a else None

    if div_p:
        style = div_p.get('style', '')
        image_urls = re.findall(r"url\('([^']+)'\)", style)
        extracted_name = extract_menu_name(image_urls[0]).strip() if image_urls else None
    else:
        extracted_name = None

    # 비교 결과 출력
    if menu_text != extracted_name:
        print(f"[{i}] ❌ 불일치")
        print(f" - 메뉴     : {menu_text}")
        print(f" - 이미지명 : {extracted_name}")
#     else:
#         print(f"[{i}] ✅ 일치 - '{menu_text}'")


[0] ❌ 불일치
 - 메뉴     : 나만의 사색피자L
 - 이미지명 : 나만의-사색피자L
[1] ❌ 불일치
 - 메뉴     : 반반 오리지널L
 - 이미지명 : 반반오리지널L
[2] ❌ 불일치
 - 메뉴     : [신메뉴]바질 콰트로치즈 피자
 - 이미지명 : 신메뉴-바질-콰트로치즈-피자
[3] ❌ 불일치
 - 메뉴     : [신메뉴]치즈수플레 미트파이 피자
 - 이미지명 : 신메뉴-치즈수플레-미트파이-피자
[4] ❌ 불일치
 - 메뉴     : 오리지널+[신메뉴]그릴드 와일드 통다리
 - 이미지명 : 오리지널-신메뉴-그릴드-와일드-통다리
[6] ❌ 불일치
 - 메뉴     : 나만의 사색피자L
 - 이미지명 : 나만의-사색피자L
[7] ❌ 불일치
 - 메뉴     : 반반 오리지널L
 - 이미지명 : 반반오리지널L
[8] ❌ 불일치
 - 메뉴     : [신메뉴]바질 콰트로치즈 피자
 - 이미지명 : 신메뉴-바질-콰트로치즈-피자
[9] ❌ 불일치
 - 메뉴     : [신메뉴]치즈수플레 미트파이 피자
 - 이미지명 : 신메뉴-치즈수플레-미트파이-피자
[10] ❌ 불일치
 - 메뉴     : 오리지널+[신메뉴]그릴드 와일드 통다리
 - 이미지명 : 오리지널-신메뉴-그릴드-와일드-통다리
[12] ❌ 불일치
 - 메뉴     : 빅파 불고기 피자
 - 이미지명 : 빅파-불고기-피자
[16] ❌ 불일치
 - 메뉴     : 프리미엄 + 사이드메뉴(택1) + 음료1.25L
 - 이미지명 : 프리미엄Ln사이드메뉴-택1n음료1250mL
[17] ❌ 불일치
 - 메뉴     : 빅파 불고기 피자
 - 이미지명 : 빅파-불고기-피자
[21] ❌ 불일치
 - 메뉴     : 프리미엄 + 사이드메뉴(택1) + 음료1.25L
 - 이미지명 : 프리미엄Ln사이드메뉴-택1n음료1250mL
[22] ❌ 불일치
 - 메뉴     : 나만의 사색피자L
 - 이미지명 : 나만의-사색피자L
[23] ❌ 불일치
 - 메뉴     : 나만의 사색피자L
 - 이미지명 : 나만의-사색피자L